# Working with Events
__________________
As of Descartes Labs 3.2 the Catalog API now supports an event notification service which allows the user to subscribe to certain types of events within the Catalog and to define actions to be taken when an event is matched by the conditions of the subscription.

This notebook serves as an introduction to Catalog Events, for a more detailed overview please visit the [API Reference](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) and [Catalog Guide](https://docs.descarteslabs.com/guides/catalog.html#working-with-events) sections in our Documentation page. 

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import (
    EventSchedule,
    EventSubscription,
    EventType,
    ScheduledEventSubscription,
    EventSubscriptionComputeTarget,
    Placeholder,
    Image,
    DataType,
    Product,
    Resolution,
    ResolutionUnit,
    SpectralBand,
)
from descarteslabs.compute import Function

In [ ]:
import sys

## Catalog Events
Every time an image or storage blob is created or updated within the Catalog, a corresponding event can be generated which is then matched against the registered subscriptions, causing the target actions specified by the matching subscriptions to be invoked.

This example will cover the basics of creating a new event by:
* Creating a new Catalog product, alongside its bands, to serve as a sample
* Defining a function which calculates and returns a mean band ratio of a single image
* Creating a Compute function which will be triggered by the upload of a new image
* Submitting a new [`EventSubscription`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/event_subscription.html#descarteslabs.catalog.EventSubscription) which will trigger the Compute function upon new image upload

_For more details on creating new products, visit [02 Creating and Managing Products](02%20Creating%20and%20Managing%20Products.ipynb)_

In [ ]:
##Setting namespace
auth = dl.auth.Auth.get_default_auth()
org = auth.payload['org']
user_hash = auth.namespace

In [ ]:
pid = f"{org or user_hash}:sample-rgb-product:{user_hash}"
pid

Creating our new product:

In [ ]:
product = Product.get(pid)

if product:
    status = product.delete_related_objects()
    if status:
        status.wait_for_completion()
    product.delete()

In [ ]:
product = Product.get_or_create(pid)
product.name = "Example RGB Product"
product.tags = ["examples"]
product.save()
product

Adding in 3 bands:

In [ ]:
for i, name in enumerate(['red', 'green', 'blue']):
    band = SpectralBand(name=name, product=product)
    band.band_index = i
    band.file_index = 0
    band.data_type = DataType.FLOAT64
    band.data_range = (0, 1)
    band.display_range = (0, 0.4)
    band.resolution = Resolution(unit=ResolutionUnit.METERS, value=30.0)
    band.save()
    print(f"Saved: {band.id}")

Defining our function, which accepts a single image ID as an input argument alongside a subscription ID, which will be discused below:

In [ ]:
def new_image_processing(image_id, subscription_id=None):
    from descarteslabs.catalog import Image
    print(f"Found {image_id}")
    print(f"Subscription ID: {subscription_id}")
    img = Image.get(image_id)
    ndarr = img.ndarray(["red", "green"])
    red = ndarr[:,:,0]
    green = ndarr[:,:,1]
    gr_ix = (green - red) / (green + red)
    return {
        "image_id": image_id, 
        "date": img.acquired.strftime("%Y-%m-%d"),
        "gr_ix": float(gr_ix.mean())
    }

Creating and saving our Compute function:

In [ ]:
major = sys.version_info.major
minor = sys.version_info.minor
image = f"python{major}.{minor}:latest"
image

In [ ]:
async_func = Function(
    new_image_processing,
    name="New Image Processing",
    image=image,
    cpus=0.25,
    memory=512,
    maximum_concurrency=20,
    timeout=300,
    retry_count=0,
)
async_func.save()
print(f"Created: {async_func.id}")

## Event Subscriptions
Below is a sample [`EventSubscription`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/event_subscription.html#descarteslabs.catalog.EventSubscription) which will trigger the newly created Compute function upon notification of a new image upload. 

The subscription takes the following inputs:
* Name, string
* [`EventType`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/types.html#descarteslabs.catalog.EventType), in this case NEW_IMAGE
* Event Namespace, which corresponds to our newly created product. This could also be a blob namespace. 
* [`Event Subscription Target`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/types.html#descarteslabs.catalog.EventSubscriptionTarget), in this case our Compute function via [`EventSubscriptionComputeTarget`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/types.html#descarteslabs.catalog.EventSubscriptionComputeTarget)

Once saved, this subscription will be active. 

In [ ]:
subscription = EventSubscription(
    name="new_image_processing",
    event_type=[EventType.NEW_IMAGE],
    event_namespace=[pid],
    targets=[
        EventSubscriptionComputeTarget(
            async_func.id, 
            Placeholder("event.detail.id"), 
            subscription_id=Placeholder("subscription.id")
        )
    ]
)
subscription.save()
subscription

Adding a new sample image to our test product:

In [ ]:
# Set any attributes that should be set on the uploaded images
image = Image(product=product, name="image1")
# Set acquired date
image.acquired = "2023-06-15"
# Set metadata
image.cloud_fraction = 0.1
image.extra_properties = {"foo": "bar"}
# Do the upload
upload = image.upload("data/rgb.tif")
# Wait for completion
upload.wait_for_completion()
upload.status

Now that a new image has been added to the product, navigate to [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) to ensure the function has received a new job input. Also check by collecting all jobs on the new function:

In [ ]:
async_func.jobs.collect()

Cleaning up:

In [ ]:
status = product.delete_related_objects()

In [ ]:
if status:
    status.wait_for_completion()
product.delete()

Deleting the subscription:

In [ ]:
subscription.delete()